# 一、爬取目标入口站点

In [1]:
import requests
import re

G_TARGET_URL = 'https://store.steampowered.com/search/?category1=998&filter=topsellers'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36'
}
response = requests.get(G_TARGET_URL, headers=headers)
content = response.text

## 测试匹配 url 的正则表达式

In [2]:
pattern = re.compile('<a href="([a-zA-z]+://[^\s]*?)".*?data-ds-appid=.*?data-ds-tagids=.*?',re.S)
results = re.findall(pattern, content)
#results

## 爬取总页数并收集每个游戏的 url 链接

- 总页数实时变化
- 爬取的详细链接都放入 GAME_URL 该链表中

In [3]:
page_compile = re.compile('page=(.*?)" onclick=',re.S)
content = response.text
pages = re.findall(page_compile, content)
max_page = pages[2]
print('total pages is :',max_page)
GAME_URL = []
pattern = re.compile('<a href="([a-zA-z]+://[^\s]*?)".*?data-ds-appid',re.S)
for page in range(int(max_page)+1):
    targe_url = G_TARGET_URL+'&page='+str(page)
    try :
        response = requests.get(targe_url, headers=headers)
        content = response.text
        results = re.findall(pattern, content)
        for url in results:
            if len(url)< 60 :
                #print("del wrong url :",url)
                results.remove(url)
                
        GAME_URL.extend(results)
        print(targe_url," is OK"+'\r',end="")
    except:
        print(targe_url," is NOT OK"+'\r',end="")

total pages is : 116


## 查看所有详细链接

In [4]:
GAME_URL

['https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/570/Dota_2/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/239140/Dying_Light/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/976310/Mortal_Kombat11/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/230410/Warframe/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/814380/Sekiro_Shadows_Die_Twice/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/271590/Grand_Theft_Auto_V/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/632360/Risk_of_Rain_2/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/sub/88801/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/477160/Human_Fall_Flat/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/859580/Imperator_Rome/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered

## 所有详细链接保存至 urls.txt

In [5]:
with open("urls.txt",'w+') as f:
    for url in GAME_URL:
        f.write(str(url))
        f.write('\n')

# 二、爬取单个游戏的详细信息

In [6]:
TARGET_URL = 'https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/'
response = requests.get(TARGET_URL, headers=headers)
content = response.text


## 获取游戏名

In [7]:
name_pattern = re.compile('<div class="apphub_AppName">(.*?)</div>',re.S)
results = re.findall(name_pattern, content)
results

['Counter-Strike: Global Offensive']

## 获取游戏公司

In [8]:
developer_pattern = re.compile(r'"developers_list".*?<a href=.*?>(.*?)</a>',re.S)
results = re.findall(developer_pattern, content)
results

['Valve']

## 获取好评等级

In [9]:
description_pattern = re.compile('<span class="game_review_summary positive" itemprop="description">(.*?)</span>',re.S)
results = re.findall(description_pattern, content)
results

['Very Positive']

## 获取评价人数

In [10]:
responsive_pattern = re.compile('<span class="responsive_hidden">.*?\\((.*?)\\).*?</span>',re.S)
results = re.findall(responsive_pattern, content)
results[1]

'3,044,830'

## 获取发行时间

In [11]:
date_pattern = re.compile('<div class="date">(.*?)</div>',re.S)
results = re.findall(date_pattern, content)
results

['21 Aug, 2012']

## 获取价格

In [12]:
price_pattern = re.compile('<div class="game_purchase_price price".*?>(.*?)</div>',re.S)
price = re.findall(price_pattern, content)
if len(price) <1:
    bs = etree.HTML(content) 
    price = bs.xpath('//div [@class="discount_prices"]/div[2]')[0].text
price[0].strip()

'Free to Play'

## 获取系统需求

In [15]:
from lxml import etree 
bs = etree.HTML(content) 
content_active = bs.xpath('//div [@class = "game_area_sys_req sysreq_content active"]/div[1]/ul/ul/li')
results= []
for i in content_active:
    results.append(i.xpath('./text()'))
results

[[' Windows® 7/Vista/XP'],
 [' Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 8750 processor or better'],
 [' 2 GB RAM'],
 [' Video card must be 256 MB or more and should be a DirectX 9-compatible with support for Pixel Shader 3.0'],
 [' Version 9.0c'],
 [' 15 GB available space']]

## 获取游戏标签

In [16]:
table_pattern = re.compile('<a href="https://store.steampowered.com/tags/.*?" class="app_tag" style=.*?>(.*?)</a>',re.S)
results = re.findall(table_pattern, content)
new_res = []
for r in results:
    new_res.append(r.strip())
new_res

['FPS',
 'Multiplayer',
 'Shooter',
 'Action',
 'Team-Based',
 'Competitive',
 'Tactical',
 'First-Person',
 'e-sports',
 'PvP',
 'Online Co-Op',
 'Military',
 'Co-op',
 'Strategy',
 'War',
 'Trading',
 'Difficult',
 'Realistic',
 'Fast-Paced',
 'Moddable']

## 获取语言种类

In [17]:
language_pattern = re.compile('<td style=.*? class="ellipsis">(.*?)</td>',re.S)
results = re.findall(language_pattern, content)
new_res = []
for r in results:
    new_res.append(r.strip())
new_res 

['English',
 'Czech',
 'Danish',
 'Dutch',
 'Finnish',
 'French',
 'German',
 'Hungarian',
 'Italian',
 'Japanese',
 'Korean',
 'Norwegian',
 'Polish',
 'Portuguese',
 'Portuguese - Brazil',
 'Romanian',
 'Russian',
 'Simplified Chinese',
 'Spanish - Spain',
 'Swedish',
 'Thai',
 'Traditional Chinese',
 'Turkish',
 'Bulgarian',
 'Ukrainian']

# 三、爬取所有游戏的详细信息并存成 csv 文件 

- 2000 多页大约耗时 25 分钟

In [18]:
NAME = []
COMPANY = []
PRAISE_LEVEL=[]
COMMENT_NUMBER = []
LANGUAGE = []
PRICE = []
ISSUE_TIME = []
SYSTEM_REQ = []
GGME_TAG = []
index = 0
session = requests.Session()
for url in GAME_URL:
#     index = index+1  #调试 10 页
#     if index>10:
#         break
    session.cookies['cookie'] = 'wants_mature_content=1; browserid=1340293676165560377; sessionid=e882a8713f7f85abfeb4452e; timezoneOffset=28800,0; _ga=GA1.2.2034098390.1556032675; _gid=GA1.2.782308328.1556032675; Steam_Language=english; app_impressions=10:80@1_5_9__412|300@1_5_9__412|20@1_5_9__412|30@1_5_9__412|40@1_5_9__412|50@1_5_9__412|60@1_5_9__412|70@1_5_9__412|130@1_5_9__412|291480@1_5_9__300|222880@1_5_9__300|10@1_5_9__300|240@1_5_9__300|10:80@1_5_9__412|300@1_5_9__412|20@1_5_9__412|30@1_5_9__412|40@1_5_9__412|50@1_5_9__412|60@1_5_9__412|70@1_5_9__412|130@1_5_9__412|291480@1_5_9__300|222880@1_5_9__300|10@1_5_9__300|240@1_5_9__300; steamCountry=CN%7Ce2a2b396cdbf0bada4bc1dd9a31f10fe; birthtime=723139201; lastagecheckage=1-0-1993; recentapps=%7B%22239140%22%3A1556039491%2C%22570%22%3A1556039076%2C%22730%22%3A1556034113%7D'
    response = session.get(url, headers=headers)
    content = response.text
    try:
        #游戏名
        name_pattern = re.compile('<div class="apphub_AppName">(.*?)</div>',re.S)
        name = re.findall(name_pattern, content)
        print("name is OK !")
        #公司
        developer_pattern = re.compile(r'"developers_list".*?<a href=.*?>(.*?)</a>',re.S)
        developer = re.findall(developer_pattern, content) 
        print("developer is OK !")
        #评论人数
        responsive_pattern = re.compile('<span class="responsive_hidden">.*?\\((.*?)\\).*?</span>',re.S)
        comment_num = re.findall(responsive_pattern, content)
        print("comment_num is OK !")
        #好评等级
        description_pattern = re.compile('<span class="game_review_summary positive" itemprop="description">(.*?)</span>',re.S)
        description = re.findall(description_pattern, content)
        print("description is OK !")
        #语言种类
        language_pattern = re.compile('<td style=.*? class="ellipsis">(.*?)</td>',re.S)
        languages = re.findall(language_pattern, content)
        new_res = []
        for r in languages:
            new_res.append(r.strip())
        print("languages is OK !")
        #价格
        price_pattern = re.compile('<div class="game_purchase_price price".*?>(.*?)</div>',re.S)
        price = re.findall(price_pattern, content)
        #打折情况
        if len(price) <1: 
            bs = etree.HTML(content) 
            price = bs.xpath('//div [@class="discount_prices"]/div[2]')[0].text
        print("price is OK !")
        #发行日期
        date_pattern = re.compile('<div class="date">(.*?)</div>',re.S)
        date = re.findall(date_pattern, content)
        print("date is OK !")
        #配置要求
        bs = etree.HTML(content) 
        content_active = bs.xpath('//div [@class = "game_area_sys_req sysreq_content active"]/div[1]/ul/ul/li')
        req= []
        for i in content_active:
            req.append(i.xpath('./text()'))
        print("req is OK !")
        #游戏标签
        tag_pattern = re.compile('<a href="https://store.steampowered.com/tags/.*?" class="app_tag" style=.*?>(.*?)</a>',re.S)
        tags = re.findall(tag_pattern, content)
        new_res = []
        for r in tags:
            new_res.append(r.strip())     
        print("tags is OK !")
        
        NAME.append(name)
        COMPANY.append(developer)
        PRAISE_LEVEL.append(description)
        if len(comment_num)>=2:
            COMMENT_NUMBER.append(comment_num[1])
        else:
            COMMENT_NUMBER.append('No user reviews')
        LANGUAGE.append(new_res)
        PRICE.append(price[0].strip())
        ISSUE_TIME.append(date)
        SYSTEM_REQ.append(req)
        GGME_TAG.append(new_res)
        print(url," is OK !")
        
    except:
        #有些网页需要登录账号查看，所以会导致获取数据失败
        print(url," is Not OK !!!!!!!!!!!!!!!!!!!!!!!!!!")

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/570/Dota_2/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/239140/Dying_Light/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/976310/Mortal_Kombat11/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
descripti

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/359550/Tom_Clancys_Rainbow_Six_Siege/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/552990/World_of_Warships/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/346110/ARK_Survival_Evolved/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/755790/Ring_of_Elysium/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comm

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/648800/Raft/?snr=1_7_7_topsellers_150_2  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/518790/theHunter_Call_of_the_Wild/?snr=1_7_7_topsellers_150_2  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/457140/Oxygen_Not_Included/?snr=1_7_7_topsellers_150_3  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/292030/The_Witcher_3_Wild_Hunt/?snr=1_7_7_topsellers_150_3  is OK !
name is OK !
developer is OK !
comment_num i

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/49520/Borderlands_2/?snr=1_7_7_topsellers_150_4  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/863550/HITMAN_2/?snr=1_7_7_topsellers_150_4  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/368070/Sniper_Ghost_Warrior_3/?snr=1_7_7_topsellers_150_4  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/816020/JUMP_FORCE/?snr=1_7_7_topsellers_150_4  is OK !
name is OK !
developer is OK !
comment_num is OK !
description i

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/644560/Mirror/?snr=1_7_7_topsellers_150_5  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/653530/Return_of_the_Obra_Dinn/?snr=1_7_7_topsellers_150_5  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/927380/Yakuza_Kiwami_2/?snr=1_7_7_topsellers_150_5  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/50300/Spec_Ops_The_Line/?snr=1_7_7_topsellers_150_5  is OK !
name is OK !
developer is OK !
comment_num is OK !
descr

...

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/934540/Pleasure_PuzzleSexy_Girls/?snr=1_7_7_topsellers_150_109  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/841430/Need_for_Synthol/?snr=1_7_7_topsellers_150_109  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/448610/Draw_Rider/?snr=1_7_7_topsellers_150_109  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/286360/Shadows_on_the_Vatican_Act_I_Greed/?snr=1_7_7_topsellers_150_109  is OK !
name is OK !
developer 

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/556010/Neon_Arena/?snr=1_7_7_topsellers_150_111  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/670280/In_Dungeon/?snr=1_7_7_topsellers_150_111  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/780760/Lots_of_Balls/?snr=1_7_7_topsellers_150_111  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/467430/Farming_6in1_bundle/?snr=1_7_7_topsellers_150_111  is OK !
name is OK !
developer is OK !
comment_num is OK !
descr

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/410670/Suits_A_Business_RPG/?snr=1_7_7_topsellers_150_112  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/551110/Wayout/?snr=1_7_7_topsellers_150_112  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/341310/Greyfox_RPG/?snr=1_7_7_topsellers_150_112  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/708150/Lost_Artifacts__Ancient_Tribe_Survival/?snr=1_7_7_topsellers_150_112  is OK !
name is OK !
developer is OK !
co

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/899370/Quick_Maths_addition_and_subtraction/?snr=1_7_7_topsellers_150_114  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/436140/Empire_of_the_Gods/?snr=1_7_7_topsellers_150_114  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/923690/Hackerexe/?snr=1_7_7_topsellers_150_114  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/717960/SoTo/?snr=1_7_7_topsellers_150_114  is OK !
name is OK !
developer is OK !
comment_nu

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/908690/BigBoy__Visual_Novel/?snr=1_7_7_topsellers_150_115  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/929840/Infinite_road/?snr=1_7_7_topsellers_150_115  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/497240/Pool_of_Death/?snr=1_7_7_topsellers_150_115  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/961270/HENTAI_CASINO/?snr=1_7_7_topsellers_150_115  is OK !
name is OK !
developer is OK !
comment_num is OK 

In [56]:
import pandas as pd
from pandas.core.frame import DataFrame
 
all_data={"name" : NAME,
   "company" : COMPANY,
   "praise level" : PRAISE_LEVEL,
   "comment number" : COMMENT_NUMBER,
   "language" : LANGUAGE,
   "price" : PRICE,
   "issue time" : ISSUE_TIME,
   "system require" : SYSTEM_REQ,
   "game tags" : GGME_TAG}#将列表转换成字典
data=DataFrame(all_data)#将字典转换成为数据框

In [57]:
# 简单清洗数据
data_copy = data
def dataFilter(category):
    s = str(category)
    s = s.replace('[','')
    s = s.replace(']','')
    s = s.replace('\'','')
    s = s.replace('¥','')
    return s

data['Name'] = data_copy['name'].apply(dataFilter)
data['Company'] = data_copy['company'].apply(dataFilter)
data['Praise Level'] = data_copy['praise level'].apply(dataFilter)
data['Comment Number'] = data_copy['comment number'].apply(dataFilter)
data['Language'] = data_copy['language'].apply(dataFilter)
data['Price'] = data_copy['price'].apply(dataFilter)
data['Issue time'] = data_copy['issue time'].apply(dataFilter)
data['System Require'] = data_copy['system require'].apply(dataFilter)
data['Game Tags'] = data_copy['game tags'].apply(dataFilter)

del data['name']
del data['company']
del data['praise level']
del data['comment number']
del data['language']
del data['price']
del data['issue time']
del data['system require']
del data['game tags']

In [59]:
# encoding 防止中文乱码
data.to_csv('all_data.csv',encoding='utf_8_sig')